Daily route and Vehicle day Summary

In [137]:
import pandas as pd
import math
import random
import itertools
from datetime import datetime, timedelta

# ── ADJUST BASE PATH ───────────────────────────────────────────────────
base_path = "/Users/sumanthmeela/Desktop/Course/Thesis/Final Work/Final_work/"

# ── LOAD INPUT CSVs ────────────────────────────────────────────────────
routes_df     = pd.read_csv(base_path + "Routes.csv", dtype={'Node_i': int, 'Node_j': int})
distances_df  = pd.read_csv(base_path + "Node_Distances.csv")
elevation_df  = pd.read_csv(base_path + "Elevation.csv")
node_to_cs_df = pd.read_csv(base_path + "Node_to_CS.csv")
cs_to_cust_df = pd.read_csv(base_path + "CS_to_Customers.csv")
weather_df    = pd.read_csv(base_path + "Weather_data.csv")

# ── BUILD LOOKUP MAPS ──────────────────────────────────────────────────
dist_map     = distances_df.set_index(['Node_i','Node_j'])['Distance_KMS'].to_dict()
elevation_df = elevation_df[elevation_df['Node_ID'].astype(str).str.isdigit()].copy()
elevation_df['Node_ID'] = elevation_df['Node_ID'].astype(int)
elev_map     = elevation_df.set_index('Node_ID')['Elevation_m'].to_dict()
cs_map       = node_to_cs_df.set_index('Node_ID')[['CS_ID','Distance_km','Ports']].to_dict('index')
cs2cust      = cs_to_cust_df.set_index(['Charging_Station_Id','Customer_node'])['Distance'].to_dict()

# ── CONSTANTS ─────────────────────────────────────────────────────────
battery_kWh              = 70
threshold_kWh            = 0.4 * battery_kWh
accel_secs               = 15                     # s to reach 50 km/h
target_mps               = 50 * 1000/3600         # ≈13.89 m/s
acceleration             = target_mps / accel_secs
signal_wait              = 15                     # s idle at each signal
customer_wait            = 400                    # s service at customer
block_km                 = 1.5                    # km per “signal block”
regen_eff                = 0.4                    # 40% regen recovery
eta                      = 0.85                   # drivetrain efficiency
charging_rate_kW_per_min = 2.0                    # kWh per minute charging

g    = 9.81  # m/s²
A    = 6.93  # m²
Cd   = 0.45  # drag coeff

# ── HELPER FUNCTIONS ───────────────────────────────────────────────────

def compute_slope(i, j):
    d_m = dist_map.get((i, j), 0.0)*1000
    return math.atan2(elev_map.get(j,0) - elev_map.get(i,0), d_m) if d_m > 0 else 0.0

def compute_drive_time(distance_km):
    d_m     = distance_km * 1000
    block_m = block_km * 1000
    accel_d = 0.5 * acceleration * accel_secs**2
    full    = int(d_m // block_m)
    rem     = d_m % block_m
    t       = 0.0
    for _ in range(full):
        cruise_t = max(0.0, (block_m - 2*accel_d) / target_mps)
        t += accel_secs + cruise_t + accel_secs + signal_wait
    if rem > 0:
        if rem <= accel_d:
            t += math.sqrt(2 * rem / acceleration)
        elif rem <= 2*accel_d:
            peak_v = math.sqrt(acceleration * rem)
            t     += 2*(peak_v/acceleration)
        else:
            rem_c = rem - 2*accel_d
            t    += accel_secs + rem_c/target_mps + accel_secs
    return t  # seconds

def compute_traction_kWh(dist_km, weight, slope, rho, fr, v_wind):
    d_m     = dist_km * 1000
    block_m = block_km * 1000
    full    = int(d_m // block_m)
    rem     = d_m % block_m
    tot_E   = 0.0
    tot_brk = 0.0

    def block_energy(d):
        E_J = br = 0.0
        accel_t = decel_t = accel_secs
        accel_d = 0.5 * acceleration * accel_t**2
        decel_d = accel_d
        cruise_d= max(0, d - accel_d - decel_d)
        cruise_t= cruise_d / target_mps
        # Acceleration
        for s in range(int(accel_t)):
            v    = acceleration * s
            F_rr = fr * weight * g * math.cos(slope)
            F_d  = 0.5 * rho * A * Cd * max(0, v - v_wind)**2
            F_sl = weight * g * math.sin(slope)
            F_ac = 1.05 * weight * acceleration
            E_J += (F_rr + F_d + F_sl + F_ac) * v
        # Cruise
        for _ in range(int(cruise_t)):
            v    = target_mps
            F_rr = fr * weight * g * math.cos(slope)
            F_d  = 0.5 * rho * A * Cd * max(0, v - v_wind)**2
            F_sl = weight * g * math.sin(slope)
            E_J += (F_rr + F_d + F_sl) * v
        # Deceleration
        for s in range(int(decel_t)):
            v    = max(0, target_mps - acceleration*s)
            F_rr = fr * weight * g * math.cos(slope)
            F_d  = 0.5 * rho * A * Cd * max(0, v - v_wind)**2
            F_sl = weight * g * math.sin(slope)
            F_ac = 1.05 * weight * -acceleration
            E_J += (F_rr + F_d + F_sl) * v
            br  += (-F_ac) * v
        return E_J, br

    for _ in range(full):
        e, b = block_energy(block_m)
        tot_E   += e
        tot_brk += b
    if rem > 0:
        e, b = block_energy(rem)
        tot_E  += e
        tot_brk += b

    # regen credit & convert J→kWh
    tot_E -= regen_eff * tot_brk
    return tot_E / (eta * 3.6e6)

# ── MAIN SIMULATION LOOP ────────────────────────────────────────────────
weather_df['Date'] = pd.to_datetime(weather_df['Date'], format="%m/%d/%y")

route_log     = []
daily_summary = []

for _, w in weather_df.iterrows():
    date     = w['Date']
    rho      = w['Air_density']
    wind_mps = w['Windspeed'] * 1000/3600
    fr       = w['Rolling_resistance_coefficent']
    P_aux    = w['Auxillary_consumption_in_kwh']

    for veh in routes_df['Vehicle'].unique():
        vr   = routes_df[routes_df['Vehicle']==veh]
        last = vr['Node_j'].iloc[-1]
        bat  = battery_kWh
        start= datetime.combine(date, datetime.strptime("08:00","%H:%M").time())
        tt   = start

        Etot = Dtot = Wtot = detours = 0
        dist_tot = 0.0

        # --- loop each segment ---
        for _, r in vr.iterrows():
            i, j   = r['Node_i'], r['Node_j']
            wgt    = r['Weight_at_node_i']
            seg_km = dist_map.get((i,j), 0.0)
            slope  = compute_slope(i, j)

            # drive + service
            dt_s   = compute_drive_time(seg_km)
            tt    += timedelta(seconds=dt_s + customer_wait)
            Dtot  += dt_s
            Wtot  += customer_wait
            dist_tot += seg_km

            # energy
            tr_kWh  = compute_traction_kWh(seg_km, wgt, slope, rho, fr, wind_mps)
            aux_kWh = P_aux * (dt_s/3600.0)
            E_kWh   = tr_kWh + aux_kWh
            bat    -= E_kWh
            Etot   += E_kWh

            # time-window compliance
            tw0  = datetime.strptime(r['Time_window_start'], "%H:%M").time()
            tw1  = datetime.strptime(r['Time_window_end'],   "%H:%M").time()
            comp = (tw0 <= tt.time() <= tw1)

            # base log entry
            entry = {
                'Date':         date,
                'Vehicle':      veh,
                'From':         i,
                'To':           j,
                'Distance_km':  round(seg_km,3),
                'Arrival_Time': tt.strftime("%H:%M:%S"),
                'Elapsed_Min':  round((tt-start).total_seconds()/60,2),
                'Battery':      round(bat,3),
                'Traction_kWh': round(tr_kWh,3),
                'Aux_kWh':      round(aux_kWh,3),
                'Energy_kWh':   round(E_kWh,3),
                'Detours':      detours,
                'CS_ID':        None,
                'CS_Dist_km':   None,
                'CS_Wait_min':  None,
                'TW_Start':     tw0,
                'TW_End':       tw1,
                'Compliant':    comp
            }

            # charging detour when needed
            if bat <= threshold_kWh and j != last and j in cs_map:
                cs_info = cs_map[j]
                ports   = cs_info['Ports']
                dcs     = cs_info['Distance_km']

                # to CS & back
                tt         += timedelta(seconds=compute_drive_time(dcs))
                bat        -= compute_traction_kWh(dcs, wgt, 0.0, rho, fr, wind_mps)
                back       = cs2cust.get((cs_info['CS_ID'], j), 0.0)
                tt         += timedelta(seconds=compute_drive_time(back))
                bat        -= compute_traction_kWh(back, wgt, 0.0, rho, fr, wind_mps)

                # stochastic queue
                p_busy     = random.random()
                patterns   = list(itertools.product([0,1], repeat=ports))
                probs      = [(p_busy**sum(p)) * ((1-p_busy)**(ports-sum(p))) for p in patterns]
                cum        = list(itertools.accumulate(probs))
                rnum       = random.random()
                chosen_pat = patterns[next(i for i,thr in enumerate(cum) if rnum <= thr)]
                free_ports = chosen_pat.count(0)
                queue_min  = random.uniform(0,30) if free_ports==0 else 0.0

                # charge
                #Made a change here changed battery_kWh to 42
                need_kWh   = 0.8 * battery_kWh - bat
                charge_min = need_kWh / charging_rate_kW_per_min
                tt        += timedelta(minutes=queue_min + charge_min)
                bat        = 0.8 * battery_kWh
                detours   += 1

                # update entry
                entry.update({
                    'Detours':     detours,
                    'CS_ID':       cs_info['CS_ID'],
                    'CS_Dist_km':  round(dcs + back,3),
                    'CS_Wait_min': round(queue_min + charge_min,2)
                })

            route_log.append(entry)

        # --- now compute day‐level compliance for this vehicle/date ---
        segs       = [r for r in route_log if r['Vehicle']==veh and r['Date']==date]
        num_segs   = len(segs)
        on_time    = sum(1 for r in segs if r['Compliant'])
        compliance = (on_time / num_segs * 100) if num_segs else 0.0

        # summary
        daily_summary.append({
            'Date':               date,
            'Vehicle':            veh,
            'Total_Distance_km':  round(dist_tot,3),
            'Final_Battery_kWh':  round(bat,3),
            'Total_Traction_kWh': round(sum(r['Traction_kWh'] for r in segs), 3),
            'Total_Aux_kWh':      round(sum(r['Aux_kWh']    for r in segs), 3),
            'Total_Energy_kWh':   round(Etot,3),
            'Drive_Time_min':     round(Dtot/60,1),
            'Wait_Time_min':      round(Wtot/60,1),
            'Charging_Stops':     detours,
            'Compliance_pct':     round(compliance,2),
            'All_TW_OK':          (on_time == num_segs),
            'Last_Arrival_Time':  tt.strftime("%H:%M:%S")
        })

# ── WRITE OUTPUTS ────────────────────────────────────────────────────────
pd.DataFrame(route_log).to_csv(base_path + "Route_Level_Summary.csv", index=False)
pd.DataFrame(daily_summary).to_csv(base_path + "Vehicle_Day_Summary.csv", index=False)


Customer removal to reach by 5:00PM

In [138]:
import itertools
from datetime import datetime, timedelta

# Reuse compute_drive_time, compute_slope, compute_traction_kWh, compute_charging_time
# from Cell 1’s environment.

# ── CONSTANTS ─────────────────────────────────────────────────────────
battery_kWh              = 70
threshold_kWh            = 0.4 * battery_kWh  # Threshold for rerouting (28 kWh)
charging_capacity_kWh     = 0.8 * battery_kWh  # Charging up to 80% (56 kWh)

# ── HELPER FUNCTION FOR FINISH TIME WITH CHARGING ─────────────────────
def finish_time_with_charging(seq, date, rho, wind_mps, fr, P_aux_kW, weight_map):
    """
    Simulate a full route sequence (including charging detours) and return
    the datetime when it finishes.
    """
    tt  = datetime.combine(date, datetime.strptime("08:00","%H:%M").time())
    bat = battery_kWh
    last = seq[-1]

    # step through each hop in sequence
    for i_node, j_node in zip(seq[:-1], seq[1:]):
        # drive + service
        dist_km = dist_map.get((i_node, j_node), 0.0)
        dt_s    = compute_drive_time(dist_km)
        tt     += timedelta(seconds=dt_s + customer_wait)

        # energy use
        slope   = compute_slope(i_node, j_node)
        tr_kWh  = compute_traction_kWh(dist_km,
                                       weight_map[i_node],
                                       slope, rho, fr, wind_mps)
        aux_kWh = P_aux_kW * (dt_s/3600.0)
        bat    -= (tr_kWh + aux_kWh)

        # charging detour if needed
        if bat <= threshold_kWh and j_node != last and j_node in cs_map:
            cs    = cs_map[j_node]
            ports = cs['Ports']
            dcs   = cs['Distance_km']

            # to CS
            tt   += timedelta(seconds=compute_drive_time(dcs))
            bat  -= compute_traction_kWh(dcs,
                                         weight_map[j_node],
                                         0.0, rho, fr, wind_mps)

            # back from CS
            back = cs2cust.get((cs['CS_ID'], j_node), 0.0)
            tt   += timedelta(seconds=compute_drive_time(back))
            bat  -= compute_traction_kWh(back,
                                         weight_map[j_node],
                                         0.0, rho, fr, wind_mps)

            # actual charging
            need = charging_capacity_kWh - bat  # Charging to 80% (56 kWh)
            wm   = compute_charging_time(need, ports)
            tt   += timedelta(minutes=wm)
            bat   = charging_capacity_kWh  # Charge to 80% capacity

    return tt

# ── BUILD PRUNING RESULTS ──────────────────────────────────────────────
pruned = []
for _, w in weather_df.iterrows():
    date     = w['Date']
    rho      = w['Air_density']
    wind_mps = w['Windspeed'] * 1000/3600
    fr       = w['Rolling_resistance_coefficent']
    P_aux    = w['Auxillary_consumption_in_kwh']
    cutoff   = datetime.combine(date, datetime.strptime("17:0","%H:%M").time())

    for veh in routes_df['Vehicle'].unique():
        vr         = routes_df[routes_df['Vehicle']==veh]
        # full loop: start@41 → all customers → return@41
        seq        = [41] + vr['Node_j'].tolist() + [41]
        weight_map = vr.set_index('Node_i')['Weight_at_node_i'].to_dict()

        # original finish time
        orig_ft = finish_time_with_charging(seq, date, rho, wind_mps, fr, P_aux, weight_map)

        # try removing minimal customers
        rem_count = 0
        removed   = []
        adj_seq   = seq

        if orig_ft > cutoff:
            customers = seq[1:-1]
            N         = len(customers)
            found     = False

            for k in range(1, N+1):
                for comb in itertools.combinations(customers, k):
                    candidate = [41] + [n for n in customers if n not in comb] + [41]
                    ft = finish_time_with_charging(candidate, date, rho, wind_mps, fr, P_aux, weight_map)
                    if ft <= cutoff:
                        rem_count = k
                        removed   = list(comb)
                        adj_seq   = candidate
                        found     = True
                        break
                if found:
                    break

        pruned.append({
            'Date':             date,
            'Vehicle':          veh,
            'Original_Finish':  orig_ft.strftime("%H:%M:%S"),
            'Made_Cutoff':      orig_ft <= cutoff,
            'Removal_Count':    rem_count,
            'Removed_Customers':removed,
            'Adjusted_Route':   adj_seq,
            'New_Finish':       finish_time_with_charging(adj_seq, date, rho, wind_mps, fr, P_aux, weight_map).strftime("%H:%M:%S")
        })

# ── SAVE PRUNED RESULTS ─────────────────────────────────────────────────
pd.DataFrame(pruned).to_csv(base_path + "Vehicle_Day_Pruned.csv", index=False)

print(f"Wrote base summary, detail, and pruned summaries.")

Wrote base summary, detail, and pruned summaries.


In [149]:
# ─────────────────────────────────────────────────────────────────────────────
# Cell 3:  Day‐level reroute with correct station handling (no KeyError)
# ─────────────────────────────────────────────────────────────────────────────

import pandas as pd
from datetime import datetime, timedelta
import ast
import itertools

# ── CONSTANTS ─────────────────────────────────────────────────────────
battery_kWh              = 70
threshold_kWh            = 0.4 * battery_kWh  # Threshold for rerouting (28 kWh)
charging_capacity_kWh    = 0.8 * battery_kWh  # Charging up to 80% capacity (56 kWh)
SERVICE_SEC              = 400   # Service time at customer
START_TIME               = datetime.strptime("08:00","%H:%M").time()
END_TIME                 = datetime.strptime("17:00","%H:%M").time()
DEPOT                    = 41

# ── SIMULATE DAY TOUR ──────────────────────────────────────────────────
def simulate_day_tour(removed_list, date, rho, wind_mps, fr, P_aux_kW, weight_map):
    """
    Returns (route_sequence, finish_datetime) with correct loc tracking.
    """
    now = datetime.combine(date, START_TIME)
    bat = battery_kWh
    loc = DEPOT
    tour = [DEPOT]

    visited = set()  # To track visited nodes and avoid revisiting
    for idx, cust in enumerate(removed_list):
        if cust in visited:
            continue  # Skip this customer if already visited
        # drive DEPOT/last_cust -> cust
        dist_km = dist_map.get((loc, cust), dist_map.get((cust, loc), 0.0))
        dt_s    = compute_drive_time(dist_km)
        now     += timedelta(seconds=dt_s + SERVICE_SEC)

        # energy use
        slope   = compute_slope(loc, cust)
        tr_kWh  = compute_traction_kWh(dist_km, weight_map[loc], slope, rho, fr, wind_mps)
        aux_kWh = P_aux_kW * (dt_s/3600.0)
        bat     -= (tr_kWh + aux_kWh)

        # update location & record
        loc = cust
        tour.append(cust)
        visited.add(cust)

        # charging detour if needed (not on last customer)
        if bat <= threshold_kWh and idx < len(removed_list)-1 and cust in cs_map:
            cs_info = cs_map[cust]
            cs_id   = cs_info["CS_ID"]
            dcs     = cs_info["Distance_km"]
            ports   = cs_info["Ports"]

            # drive to CS
            now     += timedelta(seconds=compute_drive_time(dcs))
            bat     -= compute_traction_kWh(dcs, weight_map[cust], 0.0, rho, fr, wind_mps)

            # drive back to cust
            back = cs2cust.get((cs_id, cust), 0.0)
            now  += timedelta(seconds=compute_drive_time(back))
            bat  -= compute_traction_kWh(back, weight_map[cust], 0.0, rho, fr, wind_mps)

            # charging
            need = charging_capacity_kWh - bat
            wait = compute_charging_time(need, ports)
            now  += timedelta(minutes=wait)
            bat   = charging_capacity_kWh  # Charge up to 80% capacity

            tour.append(cs_id)

    # final return to depot
    dist_km = dist_map.get((loc, DEPOT), dist_map.get((DEPOT, loc), 0.0))
    now     += timedelta(seconds=compute_drive_time(dist_km))
    tour.append(DEPOT)

    return tour, now

# ── LOAD DAY-PRUNED SUMMARY ────────────────────────────────────────────
day_pruned = pd.read_csv(base_path+"Vehicle_Day_Pruned.csv", parse_dates=["Date"])

# ── GROUP ALL REMOVED LISTS BY DATE ─────────────────────────────────────
day_grouped = (
    day_pruned
      .assign(Removed=lambda df: df["Removed_Customers"].apply(ast.literal_eval))
      .groupby("Date")["Removed"]
      .agg(lambda lists: [c for sub in lists for c in sub])
      .reset_index()
)

# ── SIMULATE EACH DAY ───────────────────────────────────────────────────
results = []
for _, row in day_grouped.iterrows():
    date     = row["Date"].date()
    removed  = row["Removed"]
    if not removed:
        results.append({
            "Date":        date,
            "Route":       [],
            "Return_Time": START_TIME.strftime("%H:%M"),
            "Possible":    True
        })
        continue

    # pull that day's weather
    w = weather_df[weather_df["Date"].dt.date == date].iloc[0]
    tour, finish = simulate_day_tour(
        removed, date,
        rho      = w["Air_density"],
        wind_mps = w["Windspeed"] * 1000/3600,
        fr       = w["Rolling_resistance_coefficent"],
        P_aux_kW = w["Auxillary_consumption_in_kwh"],  # keep same as previous code
        weight_map = routes_df.set_index("Node_i")["Weight_at_node_i"].to_dict()
    )

    # Check if the return time is within the allowable time window (5 PM cutoff)
    possible = finish <= datetime.combine(date, END_TIME)

    results.append({
        "Date":        date,
        "Route":       tour,
        "Return_Time": finish.strftime("%H:%M:%S"),
        "Possible":    possible
    })

# ── SAVE DAY-LEVEL MEGA-ROUTES ─────────────────────────────────────────
out_df = pd.DataFrame(results)
out_df.to_csv(base_path+"Day_Level_MegaRoute.csv", index=False)
print("Wrote Day_Level_MegaRoute.csv")


Wrote Day_Level_MegaRoute.csv


In [150]:
# ─────────────────────────────────────────────────────────────────────────────
# Cell 4:  If single‐truck fails, assign two trucks to finish by 17:00 minimizing total duration
# ─────────────────────────────────────────────────────────────────────────────

import itertools
from datetime import datetime, timedelta

# Reuse from previous cells:
#   DEPOT, START_TIME, END_TIME
#   dist_map, cs_map, cs2cust
#   compute_drive_time, compute_slope,
#   compute_traction_kWh, compute_charging_time, simulate_day_tour

# ── CONSTANTS ─────────────────────────────────────────────────────────
battery_kWh              = 70
threshold_kWh            = 0.4 * battery_kWh  # Threshold for rerouting (28 kWh)
charging_capacity_kWh     = 0.8 * battery_kWh  # Charging up to 80% (56 kWh)

# build a universal weight_map: Node_i → weight at start of that segment
weight_map_all = routes_df.set_index("Node_i")["Weight_at_node_i"].to_dict()

day_grouped  = day_grouped  # from Cell 4: each Date with flat Removed list

two_truck_out = []

for _, row in day_grouped.iterrows():
    date     = row["Date"].date()
    removed  = row["Removed"]
    start_dt = datetime.combine(date, START_TIME)
    cutoff   = datetime.combine(date, END_TIME)

    # 1) try single‐truck
    tour1, finish1 = simulate_day_tour(
        removed, date,
        rho       = weather_df.loc[weather_df["Date"].dt.date==date, "Air_density"].iloc[0],
        wind_mps  = weather_df.loc[weather_df["Date"].dt.date==date, "Windspeed"].iloc[0]*1000/3600,
        fr        = weather_df.loc[weather_df["Date"].dt.date==date, "Rolling_resistance_coefficent"].iloc[0],
        P_aux_kW  = weather_df.loc[weather_df["Date"].dt.date==date, "Auxillary_consumption_in_kwh"].iloc[0],
        weight_map= weight_map_all
    )
    if finish1 <= cutoff:
        # one truck suffices
        two_truck_out.append({
            "Date":           date,
            "Truck1_Route":   tour1,
            "Truck1_Return":  finish1.strftime("%H:%M:%S"),
            "Truck2_Route":   [],
            "Truck2_Return":  None,
            "Total_Duration_min": round((finish1 - start_dt).total_seconds()/60, 2),
            "Possible":       True
        })
        continue

    # 2) else brute‐force split into two tours
    best = None
    N    = len(removed)
    customers = removed[:]
    # generate all nonempty proper partitions
    for k in range(1, N//2+1):
        for combo in itertools.combinations(customers, k):
            t1_list = list(combo)
            t2_list = [c for c in customers if c not in combo]

            # sim truck1
            t1, f1 = simulate_day_tour(
                t1_list, date,
                rho       = weather_df.loc[weather_df["Date"].dt.date==date, "Air_density"].iloc[0],
                wind_mps  = weather_df.loc[weather_df["Date"].dt.date==date, "Windspeed"].iloc[0]*1000/3600,
                fr        = weather_df.loc[weather_df["Date"].dt.date==date, "Rolling_resistance_coefficent"].iloc[0],
                P_aux_kW  = weather_df.loc[weather_df["Date"].dt.date==date, "Auxillary_consumption_in_kwh"].iloc[0],
                weight_map= weight_map_all
            )
            # sim truck2
            t2, f2 = simulate_day_tour(
                t2_list, date,
                rho       = weather_df.loc[weather_df["Date"].dt.date==date, "Air_density"].iloc[0],
                wind_mps  = weather_df.loc[weather_df["Date"].dt.date==date, "Windspeed"].iloc[0]*1000/3600,
                fr        = weather_df.loc[weather_df["Date"].dt.date==date, "Rolling_resistance_coefficent"].iloc[0],
                P_aux_kW  = weather_df.loc[weather_df["Date"].dt.date==date, "Auxillary_consumption_in_kwh"].iloc[0],
                weight_map= weight_map_all
            )

            # both must finish by cutoff
            if f1 <= cutoff and f2 <= cutoff:
                total_dur = (f1 - start_dt).total_seconds()/60 + (f2 - start_dt).total_seconds()/60
                if best is None or total_dur < best["Total_Duration"]:
                    best = {
                        "Truck1_Route":     t1,
                        "Truck1_Return":    f1,
                        "Truck2_Route":     t2,
                        "Truck2_Return":    f2,
                        "Total_Duration":   total_dur
                    }

    if best:
        two_truck_out.append({
            "Date":             date,
            "Truck1_Route":     best["Truck1_Route"],
            "Truck1_Return":    best["Truck1_Return"].strftime("%H:%M:%S"),
            "Truck2_Route":     best["Truck2_Route"],
            "Truck2_Return":    best["Truck2_Return"].strftime("%H:%M:%S"),
            "Total_Duration_min": round(best["Total_Duration"],2),
            "Possible":         True
        })
    else:
        two_truck_out.append({
            "Date":             date,
            "Truck1_Route":     [],
            "Truck1_Return":    None,
            "Truck2_Route":     [],
            "Truck2_Return":    None,
            "Total_Duration_min": None,
            "Possible":         False
        })

# 3) save to CSV
df2 = pd.DataFrame(two_truck_out)
df2.to_csv(base_path + "Day_Level_TwoTruckRoutes.csv", index=False)
print(f"Wrote {len(df2)} rows to Day_Level_TwoTruckRoutes.csv")


Wrote 58 rows to Day_Level_TwoTruckRoutes.csv
